Bibliotecas

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import load_model
from keras.utils import pad_sequences
import joblib
import nltk

Importando os dados

In [16]:
dados = pd.read_csv('./dataset.csv', sep = ',', on_bad_lines='skip')

# Seleção

(explorando o dataset)

In [17]:
dados.head()

,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count
0,0,235147237,labergamim,595,48349,NaN,"Michigan, USA",@PetzOficial que lixo de atendimento.. pedido ...,2022-01-01 13:52:24+00:00,0,4,0,0
1,1,1020019844610248706,aguidaferreira4,125,2340,🚩,São Paulo,@PetzOficial @PetzOficial não entrega a COMID...,2022-01-01 04:36:27+00:00,0,2,4,0
2,2,40477476,RiikAndrade,1589,22119,Since 92-Gravo vídeos para Internet 📸@RiikAndr...,013,@PetzOficial vocês abrem amanhã?,2022-01-01 01:45:03+00:00,0,1,0,0
3,3,179667218,Rebeccamtp,529,36353,crente. nordestina. mãe de pets. tiro foto. te...,"Paraíba, Brasil",atendimento da @PetzOficial só piora,2021-12-31 15:59:26+00:00,0,1,1,0
4,4,772027307645083648,Guilhermegrosso,70,1378,NaN,"Encantado, Rio de Janeiro",@PetzOficial 83142475,2021-12-31 13:44:32+00:00,0,4,0,0


In [18]:
dados.count() 

Unnamed: 0            17015
author_id             17015
username              17015
author_followers      17015
author_tweets         17015
author_description    13750
author_location       11652
text                  17015
created_at            17015
retweets              17015
replies               17015
likes                 17015
quote_count           17015
dtype: int64

In [19]:
dados.shape

# É possível perceber que não há valores faltantes.

(17015, 13)

In [20]:
# confirmando:

dados.isna().sum()

Unnamed: 0               0
author_id                0
username                 0
author_followers         0
author_tweets            0
author_description    3265
author_location       5363
text                     0
created_at               0
retweets                 0
replies                  0
likes                    0
quote_count              0
dtype: int64

# Pré-processamento e Transformação

Fazendo a limpeza dos tweets com remoção @, links, números, emojis, caracteres etc.

In [21]:
def cleanTxt(text):
  text = text.lower()
  text = re.sub(r"@([a-zA-Z0-9_]{1,50})",'',str(text)) 
  text = re.sub(r'(?:\@|https?\://)\S+','',text)
  text = re.sub(r'\d+','', text)
  text = re.sub(r"[-()\"#/;:<>{}-~=|.?,]","",text)
  return text

dados["text_tratados"] = dados["text"].apply(cleanTxt)
dados["text_tratados"].head()

0     que lixo de atendimento pedido pago e previsã...
1       não entrega a comida dos bichinhos no prazo...
2                                   vocês abrem amanhã
3                             atendimento da  só piora
4                                                     
Name: text_tratados, dtype: object

In [22]:
def removeEmojis(text):
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002500-\U00002BEF"
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
        u"\u3030"
        "]+", re.UNICODE)
  text = emoji_pattern.sub(r'', text)
  return text

dados["text_tratados"] = dados["text_tratados"].apply(removeEmojis)
dados["text_tratados"].head()

0     que lixo de atendimento pedido pago e previsã...
1       não entrega a comida dos bichinhos no prazo...
2                                   vocês abrem amanhã
3                             atendimento da  só piora
4                                                     
Name: text_tratados, dtype: object

In [23]:
def removeStopWords(text):
    stemmer = nltk.stem.RSLPStemmer()
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    words = [stemmer.stem(i) for i in text.split() if not i in stopwords]
    return (" ".join(words))

dados["text_tratados"] = dados["text_tratados"].apply(removeStopWords)
dados["text_tratados"].head()

0    lix atend ped pag previs entreg pra segund nad...
1    entreg com bich praz aind fic enrol atend torm...
2                                           abr amanhã
3                                           atend pior
4                                                     
Name: text_tratados, dtype: object

In [24]:
dados.head()

,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count,text_tratados
0,0,235147237,labergamim,595,48349,NaN,"Michigan, USA",@PetzOficial que lixo de atendimento.. pedido ...,2022-01-01 13:52:24+00:00,0,4,0,0,lix atend ped pag previs entreg pra segund nad...
1,1,1020019844610248706,aguidaferreira4,125,2340,🚩,São Paulo,@PetzOficial @PetzOficial não entrega a COMID...,2022-01-01 04:36:27+00:00,0,2,4,0,entreg com bich praz aind fic enrol atend torm...
2,2,40477476,RiikAndrade,1589,22119,Since 92-Gravo vídeos para Internet 📸@RiikAndr...,013,@PetzOficial vocês abrem amanhã?,2022-01-01 01:45:03+00:00,0,1,0,0,abr amanhã
3,3,179667218,Rebeccamtp,529,36353,crente. nordestina. mãe de pets. tiro foto. te...,"Paraíba, Brasil",atendimento da @PetzOficial só piora,2021-12-31 15:59:26+00:00,0,1,1,0,atend pior
4,4,772027307645083648,Guilhermegrosso,70,1378,NaN,"Encantado, Rio de Janeiro",@PetzOficial 83142475,2021-12-31 13:44:32+00:00,0,4,0,0,


# Data Mining

##### Logistic Regression

In [25]:
# Save RL_Model to file in the current working directory
joblib_file = "joblib_lr_Model.pkl"  
# joblib.dump(pipe_lr, joblib_file)

# Load from file

joblib_lr_model = joblib.load(joblib_file)
joblib_lr_model

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('classifier', LogisticRegression())])

In [26]:
dados["sentimentos"] = joblib_lr_model.predict(dados["text_tratados"])

In [27]:
dados["sentimentos_nome"] = ["Positivo" if sentimento == 0 else "Negativo" for sentimento in dados["sentimentos"]]
dados.head()

,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count,text_tratados,sentimentos,sentimentos_nome
0,0,235147237,labergamim,595,48349,NaN,"Michigan, USA",@PetzOficial que lixo de atendimento.. pedido ...,2022-01-01 13:52:24+00:00,0,4,0,0,lix atend ped pag previs entreg pra segund nad...,0,Positivo
1,1,1020019844610248706,aguidaferreira4,125,2340,🚩,São Paulo,@PetzOficial @PetzOficial não entrega a COMID...,2022-01-01 04:36:27+00:00,0,2,4,0,entreg com bich praz aind fic enrol atend torm...,0,Positivo
2,2,40477476,RiikAndrade,1589,22119,Since 92-Gravo vídeos para Internet 📸@RiikAndr...,013,@PetzOficial vocês abrem amanhã?,2022-01-01 01:45:03+00:00,0,1,0,0,abr amanhã,0,Positivo
3,3,179667218,Rebeccamtp,529,36353,crente. nordestina. mãe de pets. tiro foto. te...,"Paraíba, Brasil",atendimento da @PetzOficial só piora,2021-12-31 15:59:26+00:00,0,1,1,0,atend pior,0,Positivo
4,4,772027307645083648,Guilhermegrosso,70,1378,NaN,"Encantado, Rio de Janeiro",@PetzOficial 83142475,2021-12-31 13:44:32+00:00,0,4,0,0,,0,Positivo


##### Deep Learning: LSTM.

In [34]:
# preparar tokenizer
tweets = list(dados["text_tratados"])
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tweets)

In [35]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 10213 unique tokens.


In [36]:
# integer codificar os documentos
sequences = tokenizer.texts_to_sequences(tweets)

In [47]:
# preencher documentos com um comprimento máximo de 47 palavras, igual no treinamento do modelo
X = pad_sequences(sequences, maxlen=47)

In [48]:
X.shape

(17015, 47)

In [49]:
model = load_model('lstm.h5')
model.load_weights("./LSTM_outputs/weights.05.hdf5")

In [50]:
y_probs = model.predict(X)

532/532 [==============================] - 10s 17ms/step


In [44]:
# Turn probabilities into an interger prediction
y_hat = []
for prob in y_probs:
    y_hat.append(np.argmax(prob))

In [45]:
dados["assuntos"] = y_hat
# create a dictionary to map numbers to corresponding strings
mapping = {0: 'Atendimento', 1: 'Entrega', 2: 'Outro', 3: 'Produto'}

# apply the mapping to the 'numbers' column of the DataFrame
dados['assuntos_nome'] = dados['assuntos'].map(mapping)

In [46]:
dados.head()

,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count,text_tratados,sentimentos,sentimentos_nome,assuntos,assuntos_nome
0,0,235147237,labergamim,595,48349,NaN,"Michigan, USA",@PetzOficial que lixo de atendimento.. pedido ...,2022-01-01 13:52:24+00:00,0,4,0,0,lix atend ped pag previs entreg pra segund nad...,0,Positivo,1,Entrega
1,1,1020019844610248706,aguidaferreira4,125,2340,🚩,São Paulo,@PetzOficial @PetzOficial não entrega a COMID...,2022-01-01 04:36:27+00:00,0,2,4,0,entreg com bich praz aind fic enrol atend torm...,0,Positivo,1,Entrega
2,2,40477476,RiikAndrade,1589,22119,Since 92-Gravo vídeos para Internet 📸@RiikAndr...,013,@PetzOficial vocês abrem amanhã?,2022-01-01 01:45:03+00:00,0,1,0,0,abr amanhã,0,Positivo,2,Outro
3,3,179667218,Rebeccamtp,529,36353,crente. nordestina. mãe de pets. tiro foto. te...,"Paraíba, Brasil",atendimento da @PetzOficial só piora,2021-12-31 15:59:26+00:00,0,1,1,0,atend pior,0,Positivo,0,Atendimento
4,4,772027307645083648,Guilhermegrosso,70,1378,NaN,"Encantado, Rio de Janeiro",@PetzOficial 83142475,2021-12-31 13:44:32+00:00,0,4,0,0,,0,Positivo,2,Outro
